# Submit PW calculation
#### Create a structure, kpoints, and input parameters and submit a Quantum ESPRESSO calculation

In [2]:
from aiida.backends.utils import load_dbenv, is_dbenv_loaded
if not is_dbenv_loaded():
    load_dbenv()

from aiida.orm.utils import load_node
from aiida.work.run import run
from aiida.orm.code import Code
from aiida.orm.data.base import Bool, Str
from aiida.orm.data.parameter import ParameterData
from aiida.orm.data.array.kpoints import KpointsData
from aiida.orm.data.structure import StructureData
from ase.lattice.spacegroup import crystal

#### Create a diamond cubic crystal structure

In [3]:
# Define a structure, employing the Atomic Structure Environment library
alat = 3.568
ase_definition = crystal('C', [(0,0,0)], spacegroup=227, cellpar=[alat, alat, alat, 90, 90, 90], primitive_cell=True)*2
structure = StructureData(ase=ase_definition)
structure.store()

<StructureData: uuid: 7791524c-7ea2-46fb-a8d6-6b8bf486b0dc (pk: 4938)>

#### Create the _k_-points mesh

In [4]:
# Define the k-points mesh
kpoints = KpointsData()
kpoints_mesh = [1, 1, 1]
kpoints.set_kpoints_mesh(kpoints_mesh)
kpoints.store()

<KpointsData: uuid: a45b5ab1-fabe-46b7-ba92-534983a2b3cc (pk: 4939)>

#### Define the calculation input parameters

In [12]:
# Define the pseudo potential family and input parameters for pw.x
pseudo_family = 'SSSP'
parameters = {
    'CONTROL': {
        "calculation": "scf",
        'tstress': True,
    },
    'SYSTEM': {
        'ecutwfc': 40.,
        'ecutrho': 320.,
    },
    'ELECTRONS': {
        'conv_thr': 1.e-10,
    }
}

#### Tell AiiDA which code to use

In [13]:
# Retrieve the code
codename = 'pw-5.1'
code = Code.get_from_string(codename)

In [14]:
# Create a new calculation from the code and set the inputs
calc = code.new_calc()
calc.use_structure(structure)
calc.use_pseudos_from_family(pseudo_family)
calc.use_parameters(ParameterData(dict=parameters))
calc.use_kpoints(kpoints)
calc.set_resources({"num_machines": 1})
calc.set_max_wallclock_seconds(30*60)

In [15]:
# Simply store the calculation and submit it
calc.store_all()
calc.submit()
print "Submitted a {} with pk<{}>".format(calc.__class__.__name__, calc.pk)

Submitted a PwCalculation with pk<4941>


In [24]:
!verdi calculation list

# Last daemon state_updater check: 0h:00m:19s ago (at 12:30:02 on 2017-04-20)
PK    State    Creation    Sched. state    Computer    Type
----  -------  ----------  --------------  ----------  ------

Total results: 0



In [25]:
# Load the calculation node and check if it finished correctly
calculation = load_node(calc.pk)
calculation.has_finished_ok()

True

In [26]:
# We can easily show all the attached outputs
calculation.get_outputs_dict()

{u'output_array': <ArrayData: uuid: 5cff3c1e-89dc-4ac0-9b47-a743fb2c3128 (pk: 4946)>,
 u'output_array_4946': <ArrayData: uuid: 5cff3c1e-89dc-4ac0-9b47-a743fb2c3128 (pk: 4946)>,
 u'output_kpoints': <KpointsData: uuid: 0b4db927-ad6b-4635-8ea6-d3846d33279f (pk: 4944)>,
 u'output_kpoints_4944': <KpointsData: uuid: 0b4db927-ad6b-4635-8ea6-d3846d33279f (pk: 4944)>,
 u'output_parameters': <ParameterData: uuid: 37d26b36-952a-473c-b6ed-d9b33ff9ae36 (pk: 4945)>,
 u'output_parameters_4945': <ParameterData: uuid: 37d26b36-952a-473c-b6ed-d9b33ff9ae36 (pk: 4945)>,
 u'remote_folder': <RemoteData: uuid: 93727b03-6b41-4368-b0f0-fb954af6074b (pk: 4942)>,
 u'remote_folder_4942': <RemoteData: uuid: 93727b03-6b41-4368-b0f0-fb954af6074b (pk: 4942)>,
 u'retrieved': <FolderData: uuid: c3790a16-10f1-4238-9e82-517e748b6339 (pk: 4943)>,
 u'retrieved_4943': <FolderData: uuid: c3790a16-10f1-4238-9e82-517e748b6339 (pk: 4943)>}

In [27]:
# There are convenient methods to directly access the results
print "Total energy = {} {}".format(calculation.res.energy, calculation.res.energy_units)

Total energy = -2471.82841453 eV
